<h1>
Diffusion Models
</h1>


# Brief Recap

**Diffusion models** are a class of generative models used primarily for generating data such as images. They work by gradually transforming a simple noise distribution into a data distribution through a series of steps. Essentially, they reverse a diffusion process, which is a stochastic process where data is incrementally degraded by adding noise.


They were initially introduced in a broader form of stochastic processes, but their specific application to generative modeling was popularized by the work of Sohl-Dickstein et al. with the paper "Deep Unsupervised Learning using Nonequilibrium Thermodynamics" in 2015. This work laid the foundation for understanding how to use a diffusion process in both forward and reverse directions to generate data.


# Architecture

<img src='assets/architecture.png' width=450>

This diagram illustrates the architecture of a diffusion model, highlighting both the forward and reverse diffusion processes.

1. **Input Data**: This is the original data that you want to model or generate, such as an image.

2. **Noise Schedule**: A predefined schedule that determines how much noise is added at each step in the forward diffusion process. This schedule controls the transformation from the original data to noise.

3. **Forward Diffusion Process**: In this phase, noise is progressively added to the input data according to the noise schedule, transforming it into noisy data. This simulates the degradation of data.

4. **Noisy Data**: The result of the forward diffusion process, where the original data has been converted into a noisy version.

5. **Neural Network**: A model trained to predict and reverse the noise added in the forward process. It learns to generate the original data from the noisy data.

6. **Reverse Diffusion Process**: Guided by the neural network and using the noisy data as a starting point, this process iteratively removes noise, working backwards to recreate a denoised version of the original data.

7. **Denoised Output**: The final output generated by the reverse diffusion process, ideally resembling the initial input data.

Overall, this architecture shows how diffusion models use a combination of noise addition and removal to generate new data that mimics the original input.


# Use cases

Diffusion models have a variety of use cases, particularly in generative tasks. Here are some key applications:

1. **Image Generation**: They are used to create realistic images from noise, often producing high-quality results. This includes art creation, synthetic media, and designing textures.

2. **Image Editing**: Diffusion models can be employed for tasks like inpainting (filling in missing parts of an image), super-resolution (increasing image resolution), and style transfer.

3. **Video Generation**: These models can be adapted to generate or enhance video content, maintaining temporal consistency between frames.

4. **3D Model Creation**: By extending their capabilities, diffusion models can assist in generating 3D content, used in gaming, simulations, and VR applications.

5. **Text and Language Modeling**: They can be used for text-based tasks, such as generating coherent paragraphs or dialogue.

6. **Audio Synthesis**: Generating and manipulating audio signals, such as creating music or sound effects that mimic real-world sounds.

7. **Data Augmentation**: Enhancing datasets by generating additional synthetic examples for training machine learning models.

These diverse applications illustrate the versatility and potential of diffusion models in various fields.

# **Diffusion Model Implementation**

### **Key Components**
* **Data Preprocessing:** Prepare the dataset, typically normalizing images to ensure they fit within the model's expected range.

* **Diffusion Process:**

  * **Forward Diffusion:** Incrementally adds noise to the data.
  * **Reverse Diffusion:** The model learns to remove this noise to reconstruct the data.
* **Noise Schedule:** Determines how noise is added at each step of the forward process.

* **Neural Network Architecture:** Usually a U-Net, which is effective for capturing spatial information and reconstructing images.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Example U-Net like architecture
def get_unet_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Encoding path
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)

    # Bottom
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)

    # Decoding path
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)

    outputs = layers.Conv2D(3, 1, activation='sigmoid', padding='same')(x)

    model = keras.Model(inputs, outputs)
    return model


### **Explanation**

**U-Net Architecture:**

<img src='assets/unet.jpg' width=500>

Utilizes encoding and decoding paths with skip connections to capture and reconstruct image features effectively.
  * **Training Process:** The model is optimized using a mean squared error loss, attempting to learn the denoising process.
  * **Epochs:** Set to a low number for quick demonstration; more epochs would be needed for real data.

**Use Cases**

This basic setup can be extended for advanced applications like super-resolution, style transfer, or generative tasks by adjusting the architecture and training protocols. For real implementations, consider using additional noise schedules and more complex architectures tailored to specific tasks.

In [ ]:
# Instantiate and compile the model
image_size = (128, 128, 3)
model = get_unet_model(input_shape=image_size)
model.compile(optimizer='adam', loss='mse')

# Example dummy data for training
import numpy as np
dummy_images = np.random.rand(100, 128, 128, 3)

# Train the model
model.fit(dummy_images, dummy_images, epochs=5, batch_size=10)

# Display some outputs
sample_output = model.predict(dummy_images[:5])
for img in sample_output:
    plt.imshow(img)
    plt.show()

# **Denoising Images using Diffusion**


### **Idea**
<img src='assets/denoise.jpg' width=500>

The idea is to see how a diffusion model deconstructs an image into noise and reversely constructs it back to a recognizable form, displaying the effectiveness of learned noise removal in generating high-quality images.

* **Forward Diffusion (Top Row):**

  Starts with a clear image and progressively adds noise over several steps, making the image noisier.

* **Reverse Diffusion (Bottom Row):**

  The model learns to gradually remove noise, starting from a noisy image and reconstructing it back into a clear image.
  Arrows indicate how the model refines the image at each step, effectively demonstrating its capability to denoise and generate the original content.



### **Reverse process**

<img src='assets/denoise_flow.jpg' width=500>

The reverse process of a diffusion model is illustrated above:

1. **Noise Initialization** (First Input \( t = 0 \)):
   - The model begins with an entirely noisy image.

2. **Model Processing**:
   - The model takes the noisy image and predicts a slightly denoised version.
   - This process iteratively refines the image by reducing noise step by step.

3. **Output and Feedback Loop**:
   - The output image, with reduced noise $( \text{noise} - 1 )$, becomes the input for the next iteration (\( t > 0 \)).
   - This feedback loop continues until the image is sufficiently denoised, progressively reconstructing the original content.

Essentially, the model uses iterative predictions to transform noise into a coherent image through multiple steps.



In [ ]:
import numpy as np

from tqdm.auto import trange, tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

## Data Preparation

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train[y_train.squeeze() == 1]
X_train = (X_train / 127.5) - 1.0

In [ ]:
IMG_SIZE = 32     # input image size, CIFAR-10 is 32x32
BATCH_SIZE = 128  # for training batch size
timesteps = 16    # how many steps for a noisy image into clear
time_bar = 1 - np.linspace(0, 1.0, timesteps + 1) # linspace for timesteps

In [ ]:
plt.plot(time_bar, label='Noise')
plt.plot(1 - time_bar, label='Clarity')
plt.legend()

## Image Processing

In [ ]:
def cvtImg(img):
    img = img - img.min()
    img = (img / img.max())
    return img.astype(np.float32)

def show_examples(x):
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        img = cvtImg(x[i])
        plt.imshow(img)
        plt.axis('off')

show_examples(X_train)

**Key Components**

* **Normalization**: This function normalizes the input image (img) to a range between 0 and 1.

* **Data Type:** It converts the image data type to np.float32, which is often preferred for numerical computations in machine learning.

## Noise Generation

We are going to simulate a diffusion process where an image is gradually degraded by adding noise over multiple steps. This process is essential for training diffusion models, as they learn to reverse this noise addition to generate images.


The `forward_noise` function takes an image and a timestep as input and applies Gaussian noise according to a pre-defined schedule. By repeating this process, you can progressively degrade the clarity of an image. This technique is fundamental to how diffusion models learn to generate images.

In [ ]:
def forward_noise(x, t):
    a = time_bar[t]        # base noise level
    b = time_bar[t + 1]    # next noise level

    noise = np.random.normal(size=x.shape)
    # Reshape 'a' and 'b' to add dimensions compatible with 'x'
    a = a[:, np.newaxis, np.newaxis, np.newaxis]  # Add 3 new axes
    b = b[:, np.newaxis, np.newaxis, np.newaxis]  # Add 3 new axes

    img_a = x * (1 - a) + noise * a
    img_b = x * (1 - b) + noise * b
    return img_a, img_b

def generate_ts(num):
    return np.random.randint(0, timesteps, size=num)

# t = np.full((25,), timesteps - 1) # if you want see clarity
# t = np.full((25,), 0)             # if you want see noisy
t = generate_ts(25)             # random for training data
a, b = forward_noise(X_train[:25], t)
show_examples(a)

**Key Components**

* **Input:** It takes an image (x) and a timestep (t) as input.
* **Noise Levels:** It determines the noise levels for the current timestep (a) and the next timestep (b) using the time_bar array. time_bar contains pre-calculated noise levels that decrease over time. This is the "noise schedule" you mentioned earlier.
* **Adding Noise:** Gaussian noise is added to the input image based on the noise level:

## Model Building

**Block Function**


This function defines a basic building block of the U-Net, consisting of convolutional layers, activation functions, and layer normalization. It processes both the image features (x_img) and timestep information (x_ts) to learn how noise affects the image at different stages of the diffusion process.

In [ ]:
def block(x_img, x_ts):
    x_parameter = layers.Conv2D(128, kernel_size=3, padding='same')(x_img)
    x_parameter = layers.Activation('relu')(x_parameter)

    time_parameter = layers.Dense(128)(x_ts)
    time_parameter = layers.Activation('relu')(time_parameter)
    time_parameter = layers.Reshape((1, 1, 128))(time_parameter)
    x_parameter = x_parameter * time_parameter

    # -----
    x_out = layers.Conv2D(128, kernel_size=3, padding='same')(x_img)
    x_out = x_out + x_parameter
    x_out = layers.LayerNormalization()(x_out)
    x_out = layers.Activation('relu')(x_out)

    return x_out

**`make_model` Function**

This function assembles the complete U-Net model:

* **Input Layers:** It creates input layers for the image (x_input) and the timestep (x_ts_input).
* **Timestep Embedding:** The timestep is embedded into a higher-dimensional representation to provide more context to the model about the diffusion stage.
* **Encoding Path:** The image is processed through a series of convolutional blocks, gradually downsampling the feature maps to capture larger-scale patterns.
* **Bottleneck:** A central block processes the most compressed representation of the image.
* **Decoding Path:** The feature maps are upsampled and combined with features from the encoding path using skip connections to reconstruct fine details.
* **Output Layer:** A final convolutional layer produces the denoised image.
* **Model Creation:** The function returns a Keras model object that takes the imag

In [ ]:
def make_model():
    x = x_input = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='x_input')

    x_ts = x_ts_input = layers.Input(shape=(1,), name='x_ts_input')
    x_ts = layers.Dense(192)(x_ts)
    x_ts = layers.LayerNormalization()(x_ts)
    x_ts = layers.Activation('relu')(x_ts)

    # ----- left ( down ) -----
    x = x32 = block(x, x_ts)
    x = layers.MaxPool2D(2)(x)

    x = x16 = block(x, x_ts)
    x = layers.MaxPool2D(2)(x)

    x = x8 = block(x, x_ts)
    x = layers.MaxPool2D(2)(x)

    x = x4 = block(x, x_ts)

    # ----- MLP -----
    x = layers.Flatten()(x)
    x = layers.Concatenate()([x, x_ts])
    x = layers.Dense(128)(x)
    x = layers.LayerNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Dense(4 * 4 * 32)(x)
    x = layers.LayerNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Reshape((4, 4, 32))(x)

    # ----- right ( up ) -----
    x = layers.Concatenate()([x, x4])
    x = block(x, x_ts)
    x = layers.UpSampling2D(2)(x)

    x = layers.Concatenate()([x, x8])
    x = block(x, x_ts)
    x = layers.UpSampling2D(2)(x)

    x = layers.Concatenate()([x, x16])
    x = block(x, x_ts)
    x = layers.UpSampling2D(2)(x)

    x = layers.Concatenate()([x, x32])
    x = block(x, x_ts)

    # ----- output -----
    x = layers.Conv2D(3, kernel_size=1, padding='same')(x)
    model = tf.keras.models.Model([x_input, x_ts_input], x)
    return model


model = make_model()
model.summary()

The U-Net architecture is characterized by its symmetrical encoding and decoding paths, resembling a "U" shape. Skip connections between corresponding layers in the encoding and decoding paths allow the model to preserve fine-grained details during the denoising process.

## Model Training



#### **Compilation**


The model is compiled with an Adam optimizer and a mean absolute error (MAE) loss function. During training, it learns to predict the noise added to the image at each timestep. By iteratively denoising images, it learns to reverse the diffusion process and generate new images from noise.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0008)
loss_func = tf.keras.losses.MeanAbsoluteError()
model.compile(loss=loss_func, optimizer=optimizer)

In [ ]:
def predict(x_idx=None):
    x = np.random.normal(size=(32, IMG_SIZE, IMG_SIZE, 3))

    for i in trange(timesteps):
        t = i
        x = model.predict([x, np.full((32), t)], verbose=0)
    show_examples(x)

predict()

In [ ]:
def train_one(x_img):
    x_ts = generate_ts(len(x_img))
    x_a, x_b = forward_noise(x_img, x_ts)
    loss = model.train_on_batch([x_a, x_ts], x_b)
    return loss

In [ ]:
def train(R=50):
    bar = trange(R)
    total = 100
    for i in bar:
        for j in range(total):
            x_img = X_train[np.random.randint(len(X_train), size=BATCH_SIZE)]
            loss = train_one(x_img)
            pg = (j / total) * 100
            if j % 5 == 0:
                bar.set_description(f'loss: {loss:.5f}, p: {pg:.2f}%')

#### **Training**

The training process involves repeatedly presenting the model with noisy images and their corresponding timesteps, and guiding it to predict less noisy versions. This process allows the model to learn the underlying patterns in the data and effectively reverse the diffusion process to generate high-quality images.

In [ ]:
for _ in range(10):
    train()
    # reduce learning rate for next training
    model.optimizer.learning_rate = max(0.000001, model.optimizer.learning_rate * 0.9)

    # show result
    predict()
    predict_step()
    plt.show()

Diffusion Models are a conceptually simple and elegant approach to the problem of generating data. Their State-of-the-Art results combined with non-adversarial training has propelled them to great heights, and further improvements can be expected in the coming years given their nascent status. In particular, Diffusion Models have been found to be essential to the performance of cutting-edge models like DALL-E 2.

